<a href="https://colab.research.google.com/github/Photopoint/metadata/blob/main/Photopoint_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Photographers - you ***need*** to edit this with details for all your photographers!
This script assumes that each photographer will have a main folder on the Google Drive into which they create subfolders and upload their JPEG photos.
The main folder path will be in the root folder ("My Drive") and called "Incoming". Each photographer has one line in the code below in the format:

> `"foldername": "Photographer Name",`

Getting the punctuation correct is vital. The "Photographer Name" part should be their name as they want it to appear on the watermark.

Photographers will need to have write access to "`My Drive/Incoming/foldername`" which for the example of Sally Mann would be the folder


> `https://drive.google.com/drive/u/0/my-drive/Incoming/Sally`

Images will be resized for social media. 1200 pixels (as of Feb 2023) is a good size for Facebook and Instagram limits. If that needs to be different, change the line that defines the `soc_med_size` variable.

In [83]:
# photographer details 
photographers = {
    "Ellen":  "Ellen J Rodgers",
    "HCB":    "Henri Cartier Bresson",
    "Sally":  "Sally Mann",
}



#2. Metadata definitions
Tweaks will be needed for each festival.
'%%' will be automatically replaced with the photographer's name.

In [84]:
standard_keywords = "%%,Beltane Fire Society,Beltane,Beltane 2023,Beltane2023,Beltane Fire Festival,Beltane Fire Festival 2022,Edinburgh,Edinburgh Beltane,Edinburgh Beltane 2023"
standard_headline = "Copyright %% for Beltane Fire Society."
standard_caption = "Copyright %% for Beltane Fire Society. All Rights Reserved. www.beltane.org / www.facebook.com/beltanefiresociety"
standard_terms = "All Rights Reserved"

#3. Drive paths and Watermark definitions (plus a couple of control things)
These will probably be fine, but feel free to tweak them if desired.

In [85]:
# where the incoming and processed photos will be on Google Drive
mydrive = "My Drive/Photopoint Drive/My Drive"
incoming_folder = "Member Uploads"
processed_folder = "Processed Photos"

# the size, in pixels, of the longest edge for social media images
soc_med_size = 1200

# the text of the watermark. '%%' will automatically be replaced with the photographer's name
watermark_template = "Photo (c) %% for the Beltane Fire Society"

# the size, in points, of the watermark text
watermark_size = 16

# change this to False if you don't want to see progress details
show_progress = True

#4. The code.
Nothing should need to be changed beyond this point! Please be sure you know what you're doing if you do make changes.


In [86]:
from genericpath import isfile
!wget "https://fonts.google.com/download?family=Roboto+Condensed" -O font.zip
!unzip -u font.zip RobotoCondensed-BoldItalic.ttf
!mv RobotoCondensed-BoldItalic.ttf /usr/share/fonts/truetype/
!pip install piexif
import PIL
from PIL import Image, ImageDraw, ImageFont
import piexif
import io
import os
import string
import datetime
from random import choice
from google.colab import drive

font_path = "/usr/share/fonts/truetype/RobotoCondensed-BoldItalic.ttf"

def random_chars(n):
	return ''.join([choice(string.ascii_letters + string.digits) for i in range(n)])
 
def filename_from_exif(image, photographer):
  data = piexif.load(image.info["exif"])
  exif = data["Exif"]
  key = None
  if piexif.ExifIFD.DateTimeOriginal in exif:
    key = piexif.ExifIFD.DateTimeOriginal
  elif piexif.ExifIFD.DateTimeDigitized in exif:
    key = piexif.ExifIFD.DateTimeDigitized
  if key:
    date_from_exif = exif[key].decode("utf-8")
  else:
    date_from_exif = "1999:12:31 23:59:59"
  
  date_str = datetime.datetime.strptime(date_from_exif,"%Y:%m:%d %H:%M:%S").strftime("%Y%m%d%H%M%S")
  rand_chars = random_chars(6)
  return f"{date_str} {photographer} {rand_chars}"

def progress(text):
  if show_progress:
    print(text)

def test_and_make_dir(path):
  ''' check if path exists and is a directory
      create it if it doesn't exist
      returns True if the directory had to be created
  '''
  if os.path.exists(path):
    if not os.path.isdir(path):
      raise NotADirectoryError(path)
  else:
    progress(f"Creating folder {path}")
    os.makedirs(path)
    return True
  return False

def file_from_path(path):
  return path[path.rfind('/'):]

def parent_dir(path):
  return path[:path.rfind('/')]

def to_processed_name(path):
  return path.replace(incoming_path,processed_path,1)

def add_watermark(image, photographer):
    # Get image width and height
    width, height = image.size

    # Create an ImageDraw object
    draw = ImageDraw.Draw(image)

    # Load the font
    font = ImageFont.truetype(font_path, watermark_size)

    w_text = watermark_template.replace("%%",photographer)
    # Calculate the size of the text
    text_width, text_height = draw.textsize(photographer, font)

    # Calculate the position of the text
    x = width//2 - text_width//2
    y = height - text_height - 20

    # Add the text to the image with a border for readability
    draw.text((x, y), photographer, font=font, fill="#222")
    draw.text((x, y+2), photographer, font=font, fill="#222")
    draw.text((x+2, y), photographer, font=font, fill="#222")
    draw.text((x+2, y+2), photographer, font=font, fill="#222")
    draw.text((x+1, y+1), photographer, font=font, fill="#ed6")

    return image

def save_image_to_drive(image, file_id, content_type):
    # Save the image with the watermark to a memory buffer
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG")


def resize_image(image):
    # Calculate the aspect ratio
    aspect_ratio = image.width / image.height

    # Resize the image to be soc_med_size pixels on the long edge
    if aspect_ratio > 1:
        new_width = soc_med_size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = soc_med_size
        new_width = int(new_height * aspect_ratio)

    return image.resize((new_width, new_height), resample=PIL.Image.LANCZOS)


def find_files(path, ext):
  files = []
  for r, d, f in os.walk(path):
    for file in f:
      progress(f"Considering {file}")
      if file.endswith(ext):
        files.append(os.path.join(r, file))
  return files

def process_image(file, dest_dir, photographer):
  img = Image.open(file)
  
  if "exif" not in img.info:
    progress(f"Failed to process {file} - missing EXIF data")
    return

  orig_filename = file_from_path(file)

  photographer_name = photographers[photographer]
  w_text = watermark_template.replace("%%", photographer_name)

  socmed_image = resize_image(img.copy())
  watermarked_socmed_image = add_watermark(socmed_image, w_text)

  watermarked_archive_image = add_watermark(img.copy(), w_text)

  newname = filename_from_exif(img, photographer_name)

  # save both watermarked images and the original to the Processed folder
  watermarked_archive_image.save(f"{dest_dir}/ARCHIVE {newname}.jpg")
  watermarked_socmed_image.save(f"{dest_dir}/SOCMED {newname}.jpg")
  moved_path = f"{dest_dir}/{orig_filename}"
  img.save(moved_path)
  if os.path.isfile(moved_path):
    os.remove(file)
    progress(f"Processed {file}")

# jpeg_files = find_files(path, '.jpeg')
# print(jpeg_files)

progress("About to start")
# Main function
if __name__ == "__main__":

    progress("Here we go!")

    # Path to the font
    

    base_path = '/content/drive'
    my_drive_path = f'{base_path}/{mydrive}'
    incoming_path = f'{my_drive_path}/{incoming_folder}'
    processed_path = f'{my_drive_path}/{processed_folder}'

    drive.mount(base_path, force_remount=True)

    dir_cache = {}
    image_count = {}
    # go through each photographer in turn
    for photographer in photographers:
        progress(f"Doing {photographer}")
        name_text = photographers[photographer]

        test_and_make_dir(f"{processed_path}/{photographer}")

        image_count[photographer] = 0

        # Process each file
        for file in find_files(f"{incoming_path}/{photographer}",'.jpg'):
            progress(f"Found {file}")

            processed_dir = to_processed_name(parent_dir(file))
            if processed_dir not in dir_cache:
              progress(f"Need to create {processed_dir}")
              if test_and_make_dir(processed_dir):
                progress("Done!")
                dir_cache[processed_dir] = True
              else:
                progress("test_and_make_dir failed?")

            process_image(file, processed_dir, photographer)
            image_count[photographer] += 1

            # Get the subfolder "Archive" if it exists, otherwise create one
            # archive_folder = get_or_create_subfolder(file["parents"][0], "Archive")

    # show stats
    for photographer in photographers:
      progress(f"Processed {image_count[photographer]} images for {photographer}")
      

--2023-02-15 16:38:38--  https://fonts.google.com/download?family=Roboto+Condensed
Resolving fonts.google.com (fonts.google.com)... 173.194.193.100, 173.194.193.113, 173.194.193.101, ...
Connecting to fonts.google.com (fonts.google.com)|173.194.193.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘font.zip’

font.zip                [ <=>                ] 540.00K  --.-KB/s    in 0.03s   

2023-02-15 16:38:38 (15.5 MB/s) - ‘font.zip’ saved [552961]

Archive:  font.zip
  inflating: RobotoCondensed-BoldItalic.ttf  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
About to start
Here we go!
Mounted at /content/drive
Doing Ellen
Considering 20161031-samhuinn-20161031-6535-Edit.jpg
Considering 20161031-samhuinn-20161031-6216.jpg
Considering 20161031-samhuinn-20161031-6448.jpg
Considering 20161031-samhuinn-20161031-6458.jpg
Considering 20161031-samhuinn-20161031-6415.jpg
Cons